# Vectorization and Array Operations with NumPy

-----
#### Written for the CBC Workshop (May 2024)

#### John Stachurski
-----

[NumPy](https://numpy.org/) is the standard library for numerical array operations in Python.

This notebook contains a very quick introduction to NumPy.

(Although the syntax and some reference concepts differ, the basic framework is similar to Matlab.)

We use the following imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## NumPy arrays

Let's review the basics of NumPy arrays.

### Creating arrays

Here are a few ways to create arrays:

In [4]:
a = (10.0, 20.0)   # Let's start with a native Python tuple
type(a)

tuple

In [5]:
a = np.array(a)   # Create a NumPy array from Python tuple
type(a)

numpy.ndarray

In [7]:
a = np.array((10, 20), dtype='float64')  # Specify data type 
a

array([10., 20.])

In [8]:
a = np.linspace(0, 10, 5)
a

array([ 0. ,  2.5,  5. ,  7.5, 10. ])

In [9]:
a = np.ones(3)
a

array([1., 1., 1.])

In [10]:
a = np.zeros(3)
a

array([0., 0., 0.])

In [11]:
a = np.random.randn(4)
a

array([0.86544028, 0.43743189, 0.23293253, 0.27260297])

In [12]:
a = np.random.randn(2, 2)
a

array([[ 1.66311339, -1.12944551],
       [ 0.44847964,  0.95503339]])

In [13]:
b = np.zeros_like(a)
b

array([[0., 0.],
       [0., 0.]])

In [15]:
c = np.empty_like(a)
c

array([[ 1.66311339, -1.12944551],
       [ 0.44847964,  0.95503339]])

### Reshaping

In [18]:
a = np.random.randn(2, 2)
a

array([[ 0.17583754, -0.03386183],
       [ 0.39340515,  0.2191531 ]])

In [19]:
a.shape

(2, 2)

In [20]:
np.reshape(a, (1, 4))

array([[ 0.17583754, -0.03386183,  0.39340515,  0.2191531 ]])

In [21]:
np.reshape(a, (4, 1))

array([[ 0.17583754],
       [-0.03386183],
       [ 0.39340515],
       [ 0.2191531 ]])

### Array operations

Standard arithmetic operators are pointwise:

In [22]:
a

array([[ 0.17583754, -0.03386183],
       [ 0.39340515,  0.2191531 ]])

In [23]:
b

array([[0., 0.],
       [0., 0.]])

In [24]:
a + b

array([[ 0.17583754, -0.03386183],
       [ 0.39340515,  0.2191531 ]])

In [25]:
a * b  # pointwise multiplication

array([[ 0., -0.],
       [ 0.,  0.]])

In [26]:
a**b

array([[1., 1.],
       [1., 1.]])

### Matrix multiplication

In [27]:
a @ b  

array([[0., 0.],
       [0., 0.]])

In [28]:
np.ones(3) @ np.zeros(3)  # inner product

0.0

### Reductions

There are various functions for acting on arrays, such as

In [29]:
np.mean(a)

0.1886334898422808

In [30]:
np.sum(a)

0.7545339593691232

These operations have an equivalent OOP syntax, as in

In [31]:
a.mean()

0.1886334898422808

In [32]:
a.sum()

0.7545339593691232

These operations also work on higher-dimensional arrays:

In [33]:
a = np.linspace(0, 3, 4).reshape(2, 2)
a

array([[0., 1.],
       [2., 3.]])

In [34]:
a.sum(axis=0)  # sum columns

array([2., 4.])

In [35]:
a.sum(axis=1)  # sum rows

array([1., 5.])

### Broadcasting

When possible, arrays are "streched" across missing dimensions to perform array operations.

For example,

In [36]:
a = np.zeros((3, 3))
a

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [37]:
b = np.array((1.0, 2.0, 3.0))
b = np.reshape(b, (1, 3))
b

array([[1., 2., 3.]])

In [38]:
a + b

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

In [39]:
b = np.reshape(b, (3, 1))
b

array([[1.],
       [2.],
       [3.]])

In [40]:
a + b

array([[1., 1., 1.],
       [2., 2., 2.],
       [3., 3., 3.]])

For more on broadcasting see [this tutorial](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html).

### Ufuncs

Many NumPy functions can act on either scalars or arrays.

When they act on arrays, they act pointwise (element-by-element).

These kinds of functions are called **universal functions** or **ufuncs**.

In [41]:
np.cos(np.pi)

-1.0

In [42]:
a = np.random.choice((0, np.pi), 6).reshape(2, 3)
a

array([[3.14159265, 0.        , 3.14159265],
       [0.        , 3.14159265, 0.        ]])

In [43]:
np.cos(a)

array([[-1.,  1., -1.],
       [ 1., -1.,  1.]])

Some user-defined functions will be ufuncs, such as

In [44]:
def f(x):
    return np.cos(np.sin(x))

In [45]:
f(a)

array([[1., 1., 1.],
       [1., 1., 1.]])

But some are not:

In [46]:
def f(x):
    if x < 0:
        return np.cos(x)
    else:
        return np.sin(x)

In [47]:
f(a)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

If we want to turn this into a vectorized function we can use `np.vectorize`

In [48]:
f_vec = np.vectorize(f)

Let's test it, and also time it.

In [49]:
a = np.linspace(0, 1, 10_000_000)
%time f_vec(a)

CPU times: user 5.05 s, sys: 220 ms, total: 5.27 s
Wall time: 5.28 s


array([0.00000000e+00, 1.00000010e-07, 2.00000020e-07, ...,
       8.41470877e-01, 8.41470931e-01, 8.41470985e-01])

This is pretty slow.

Here's a version of `f` that uses NumPy functions to create a more efficient ufunc.

In [50]:
def f(x):
    return np.where(x < 0, np.cos(x), np.sin(x))

In [51]:
%time f(a)

CPU times: user 98.5 ms, sys: 29.1 ms, total: 128 ms
Wall time: 126 ms


array([0.00000000e+00, 1.00000010e-07, 2.00000020e-07, ...,
       8.41470877e-01, 8.41470931e-01, 8.41470985e-01])

Moral of the story: Don't use `np.vectorize` unless you have to.

(There are good alternatives, which we will discuss soon.)

### Mutability

NumPy arrays are mutable (can be altered in memory).

In [52]:
a = np.array((10.0, 20.0))
a

array([10., 20.])

In [53]:
a[0] = 1

In [54]:
a

array([ 1., 20.])

In [55]:
a[:] = 42

In [56]:
a

array([42., 42.])

**All names** bound to an array have equal rights.

In [57]:
a

array([42., 42.])

In [58]:
b = a  # bind the name b to the same array object

In [59]:
id(a)

134434427105808

In [60]:
id(b)

134434427105808

In [61]:
b[0] = 1_000

In [62]:
b

array([1000.,   42.])

In [63]:
a

array([1000.,   42.])

## Vectorizing loops

### Accelerating slow loops

In scripting languages, native loops are slow:

In [64]:
n = 10_000_000
x_vec = np.linspace(0.1, 1.1, n)

Let's say we want to compute the sum of of $\cos(2\pi / x)$ over each $x$ in the array.

In [66]:
%%time
current_sum = 0.0
for x in x_vec:
    current_sum += np.cos(2 * np.pi / x)

CPU times: user 7.1 s, sys: 1.53 ms, total: 7.1 s
Wall time: 7.13 s


The reason is that Python, like most high level languages is dynamically typed.

This means that the type of a variable can freely change.

Moreover, the interpreter doesn't compile the whole program at once, so it doesn't know when types will change.

So the interpreter has to check the type of variables before any operation like addition, comparison, etc.

Hence there's a lot of fixed cost for each such operation

The code runs much faster if we use **vectorized** expressions to avoid explicit loops.

In [68]:
%%time
np.sum(np.cos(2 * np.pi / x_vec))

CPU times: user 114 ms, sys: 13 ms, total: 127 ms
Wall time: 126 ms


1352487.12437786

Now high level overheads are paid *per array rather than per float*.

### Implict Multithreading


Recent versions of Anaconda are compiled with Intel MKL support, which accelerates NumPy operations.

Watch system resources when you run this code.  

(For example, install `htop` (Linux / Mac), `perfmon` (Windows) or another system load monitor and set it running in another window.)

In [70]:
n = 20
m = 1_000
for i in range(n):
    X = np.random.randn(m, m)
    λ = np.linalg.eigvals(X)

You should see all your cores light up.  With MKL, many matrix operations are automatically parallelized.